### Importing Libraries

In [16]:
import pandas as pd
import numpy as np
import os
from googletrans import Translator
from joblib import Parallel, delayed
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

### List files in data directory

In [91]:
ip = '../data/'

files = os.listdir(path = ip)

print(files)

['items.csv', 'item_categories.csv', 'test.csv', 'sample_submission.csv', 'sales_train.csv', 'shops.csv']


### Read all files

In [92]:
%%time

data = dict()
for file in files:
    data[file[:-4]] = pd.read_csv(ip + file)
    
data['sales_train']['date'] = pd.to_datetime(data['sales_train']['date'], format = '%d.%m.%Y')

CPU times: user 1.69 s, sys: 104 ms, total: 1.8 s
Wall time: 1.79 s


In [93]:
### create temporal features

In [94]:
%%time

data['sales_train']['year'] = data['sales_train']['date'].dt.year
data['sales_train']['month'] = data['sales_train']['date'].dt.month
data['sales_train']['dow'] = [datetime.strftime(i, '%A') for i in data['sales_train']['date']]

CPU times: user 26.4 s, sys: 111 ms, total: 26.6 s
Wall time: 26.6 s


In [95]:
data['sales_train']['mkey'] = [f"{i}_{j}" 
                               for i, j in zip(data['sales_train']['shop_id'], data['sales_train']['item_id'])]

In [96]:
data['sales_train'].sample(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,dow,mkey
1829233,2014-07-19,18,13,6497,563.74,1.0,2014,7,Saturday,13_6497
63913,2013-01-02,0,30,5612,3990.00,1.0,2013,1,Wednesday,30_5612
1686377,2014-05-31,16,38,6702,399.50,1.0,2014,5,Saturday,38_6702
2629476,2015-05-26,28,28,18047,299.00,1.0,2015,5,Tuesday,28_18047
2241304,2014-12-22,23,38,1569,2999.00,1.0,2014,12,Monday,38_1569
489317,2013-05-17,4,54,17273,98.00,1.0,2013,5,Friday,54_17273
512392,2013-05-13,4,7,20949,5.00,4.0,2013,5,Monday,7_20949
2880516,2015-09-27,32,25,16287,199.00,1.0,2015,9,Sunday,25_16287
1709661,2014-06-27,17,27,15458,459.00,1.0,2014,6,Friday,27_15458
911842,2013-09-05,8,30,21427,149.00,1.0,2013,9,Thursday,30_21427


In [97]:
len(np.unique(data['sales_train']['mkey']))

424124

In [98]:
#### view overall sales pattern at monthly level

In [99]:
from plotutils import dualaxislineplot

In [100]:
aggregations = {
    'item_cnt_day':'sum',
    'mkey':pd.Series.nunique
}

grouped = data['sales_train'].groupby(['year','month'], as_index=False).agg(aggregations)

grouped['YYYYMM'] = [f"_{i}{str(j).zfill(2)}" for i, j in zip(grouped.year, grouped.month)]

dualaxislineplot(data=grouped, x="YYYYMM", primary_y="item_cnt_day", secondary_y="mkey", title='sales')

In [102]:
corr = data['sales_train'].corr()
corr.style.background_gradient(cmap='coolwarm')

,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month
date_block_num,1.000000,0.019273,0.009356,0.095010,0.009402,0.928280,0.243939
shop_id,0.019273,1.000000,0.029396,-0.024034,-0.005230,0.011805,0.020574
item_id,0.009356,0.029396,1.000000,-0.134104,0.016650,0.010340,-0.002034
item_price,0.095010,-0.024034,-0.134104,1.000000,0.011197,0.080192,0.044049
item_cnt_day,0.009402,-0.005230,0.016650,0.011197,1.000000,0.002699,0.018014
year,0.928280,0.011805,0.010340,0.080192,0.002699,1.000000,-0.134203
month,0.243939,0.020574,-0.002034,0.044049,0.018014,-0.134203,1.000000


In [103]:
data['sales_train'].head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,dow,mkey
0,2013-01-02,0,59,22154,999.00,1.0,2013,1,Wednesday,59_22154
1,2013-01-03,0,25,2552,899.00,1.0,2013,1,Thursday,25_2552
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1,Saturday,25_2552
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1,Sunday,25_2554
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,Tuesday,25_2555


In [104]:
data['sales_train'].groupby(['mkey'], as_index=False).agg({
    'date':'count'
}).sort_values(by='date', ascending=False)

,mkey,date
146521,28_20949,867
335526,53_20949,863
182447,31_20949,863
89027,22_20949,848
244951,42_20949,834
...,...,...
298697,4_12781,1
298698,4_12793,1
109458,25_17080,1
109467,25_17092,1


In [118]:
data['sales_train'][(data['sales_train']['item_id'] == 13525) & (data['sales_train']['item_price'] == 21)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,dow,mkey
1823800,2014-07-30,18,51,13525,21.0,1.0,2014,7,Wednesday,51_13525
1828248,2014-07-29,18,10,13525,21.0,1.0,2014,7,Tuesday,10_13525
1904180,2014-08-25,19,51,13525,21.0,1.0,2014,8,Monday,51_13525


In [125]:
df = data['sales_train'].copy()
print(df[(df.item_id==13525) & (df.shop_id==51)])
del df

              date  date_block_num  shop_id  item_id  item_price  \
1823800 2014-07-30              18       51    13525        21.0   
1904180 2014-08-25              19       51    13525        21.0   

         item_cnt_day  year  month        dow      mkey  
1823800           1.0  2014      7  Wednesday  51_13525  
1904180           1.0  2014      8     Monday  51_13525  
